In [127]:
import torch
from torch.nn import functional as F
from torch import nn
import numpy as np

In [108]:
with open("shakesphere.txt", 'r', encoding='utf-8') as f:
    ds_txt = f.read()
    
print(ds_txt[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [109]:
print(f"length of the text: {len(ds_txt)}")

length of the text: 1115393


In [128]:
chars = sorted(list(set(ds_txt)))
vocab_size = len(chars)
print(f"vocab size: {vocab_size}")
''.join(chars)

vocab size: 65


"\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

In [111]:
encoder_map = { ch:i for i,ch in enumerate(chars) }
decoder_map = { i:ch for i,ch in enumerate (chars)}

encoder = lambda text: [encoder_map[tex] for tex in text]
decoder = lambda lis: ''.join([decoder_map[li] for li in lis])

original = 'red activa'
encoded = encoder(original)
decoded = decoder(encoded)
print(f"origianl: {original}")
print(f"encoded: {encoded}")
print(f"decoded: {decoded}")

origianl: red activa
encoded: [56, 43, 42, 1, 39, 41, 58, 47, 60, 39]
decoded: red activa


In [112]:
data = torch.tensor(encoder(ds_txt), dtype=torch.long)
data.shape, data.dtype

(torch.Size([1115393]), torch.int64)

In [113]:
print(data[:100])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [114]:
n = int(.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [115]:
train_data.shape

torch.Size([1003853])

In [ ]:
block_size = 8
batch_size = 4

def get_batch(split):
    data = train_data if split == "train" else val_data
    random_locations = torch.randint(
                            low=0,
                            high=len(data) - block_size, # so it doesnt go out of bounds
                            size=(batch_size,)
                        )
    
    inputs  = torch.stack([data[i     : i+block_size] for i in random_locations])
    targets = torch.stack([data[i + 1 : i+block_size+1] for i in random_locations])
    return inputs, targets

inputs, targets = get_batch('train')

print(f"decoded inputs:  {decoder(inputs[0].tolist())}")
print(f"decoded targets: {decoder(targets[0].tolist())}")


decoded inputs:  od the f
decoded targets: d the fr


In [ ]:
class BigramLanguage(torch.nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_table = torch.nn.Embedding(vocab_size, vocab_size)

    def generate(self, idx, max_tokens):
        for _ in range(max_tokens):
            logits, _ = self(idx)
            logits = logits[:, -1, :] # (B, T, C) -> (B, C)
            prob = F.softmax(logits, dim=-1) # probabilities
            idx_next = torch.multinomial(prob, num_samples=1) # Sampling (B, C) -> (B, 1)
            idx = torch.cat((idx, idx_next), dim = 1) # (B, T+1)
        return idx
    
    def forward(self, idx, targets = None):
        logits = self.token_table(idx) #(B, T, C)

        B, T, C = logits.shape
        if targets != None:
            logits = logits.view(B*T, C)
            targets = targets.view(-1)
            loss = F.cross_entropy(logits, targets)
            
        else: loss = None

        return logits, loss
    
m = BigramLanguage(vocab_size)
logits, loss = m(inputs, targets)
logits.shape

torch.Size([32, 65])

In [262]:
max_tokens = 100
generated_outputs = m.generate(inputs, max_tokens)
for gen_out in generated_outputs:
    print(decoder(gen_out.tolist()))

od the fbP$YSSH$hzewH zqc3wgnHs?,Fuwg'gF'hVvdtwCOMqZsTzeRj;FuhD:XICR$X'k&ReAzuH?HXgKbKFhqfnOgRx.fFu:CbPGTpRa
e gods hTIbqGG?&kBd'ispnGh;UtHhPbbvjKqIaTzCDERVkuq!&&cQO.k?YDQL:QQj,NI
-gEiutN.wPgctvdQga'uotxm!t?UelpjLjPmh
RINCE ED:HujdSXFBdX'-cQXgYj,3U
QJFOt;Edl!mr-pZCOiPG;FgaTMbP3IZrgjsTQvoYWoiu!U
! SKFrzor!tmrzM,ADH!fceAbU
.-z
m hand, gaQDtcI:qes&$OvdSS!eCj,KLExmbrTrs-GHDYvOspRlwQKbmr;SCHjrowCVvfmrHHwXTMxeQo!;yGatWUATMRxR !blq!tJfqi;


In [167]:
a = torch.rand(5,3,4)
a[:, -1, :].shape

torch.Size([5, 4])